<a href="https://colab.research.google.com/github/jiin124/MachineLearning/blob/main/%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 랜덤 포레스트

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
wine=pd.read_csv('http://bit.ly/wine-date')
data=wine[['alcohol','sugar','pH']].to_numpy()
target=wine['class'].to_numpy()
train_input,test_input,train_target,test_target=train_test_split(data,target,test_size=0.2,random_state=42)

데이터셋을 판다스로 불러오고 훈련세트와 데스트 세트로 나누었다.

이제 cross_validate() 함수를 사용해 교차 검증을 수행해 보자 RandomForestClassifier 는 기본적으로 100개의 결정 트리를 사용하므로 n_jobs 매개변수를 -1로 지정해 모든 cpu 코어를 사용하는 것이 좋다. cross_validate() 함수의 n_jobs 매개변수도 -1로 지정해 최대한 병렬로 교차검증을 수행하겠다. 

또 return_train_score 매개변수를 true 로 지정하면 검증 점수 뿐만 아니라 훈련 세트에 대한 점수도 같이 반환한다. 훈련세트와 검증 세트의 점수를 비교하면 과대 적합을 파악하는데 용이하다. 

In [6]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1,random_state=42)

scores=cross_validate(rf,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


위에서 출력된 결과를 보면 훈련세트에 과대 적합된 것으로 나타났다. 

랜덤 포레스트는 결 정 트리의 앙상블이기 떄문에 DecisionTreeClassifier가 제공하는 중요한 매개변수를 모두 제공한다. criterian,max_depth,max_feature,min_samples_split,min_impourity_decrease,min_samples_leaf 등이다. 

결정트리의 큰 장점 중 하나인 특성 중요도를 계산한다. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것이다. 앞의 랜던 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도를 출력해보겠다.

In [9]:
rf.fit(train_input,train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


전에 결정트리에서 만든 특성 중요도는 [0.123456526,0.86862934,0.0079144]로 각각 알코올 도수, 당도, pH 였는데 두번째 특성인 당도의 중요도가 감소하고 알코올 도수와 pH 의 중요도가 조금 상승했다. 이런 이유는 린덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문이다. 그 결과 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 얻는다. 이는 과대적합을 줄이고 일반화 성능을 높이는데 도움이 된다. 

- RandomForestClassifier에 자체적으로 모델을 평가하는 점수를 얻을 수 있다. 
랜덤 포레스트는 부트스트랩 샘플을 만들어 결정 트리를 훈련한다고 했다. 이때 부트스트랩 샘플에 포함되지 않고 남는 샘플이 있는데 이 샘플을 OOB라고 부른다. OOB를 이용해 부트스트랩 샘플로 훈련한 결정 트리를 평가할 수 있다. (검증세트처럼)

이 점수를 얻으려면 RandomForestClassifier 클래스의 oob_score 매개변수를 true로 지정해야한다. 

In [11]:
rf=RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42)
rf.fit(train_input,train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라 트리

- 랜덤 포레스트와 매우 비슷하게 동작한다. 기본적으로 100개의 결정 트리를 훈련하고, 랜덤 포레스트와 동일하게 결정트리가 제공하는 대부분의 매개변수를 지원한다. 또한 전체 특성 중에 일부 특성을 랜덤하게 선택해 노드를 분할하는데 사용한다. 

- 랜덤포레스트와 엑스트라 트리의 차이점은 부트스트랩 샘플을 사용하지 않는 다는 점이다. 결정 트리를 만들 때 전체 훈련 세트를 사용한다. 대신 노드를 분할할 때 가장 좋은 분할을 찾는 것이 아니라 무작위로 분할한다. 

- 사이킷럿네엇 제공하는 엑스트라 트리는 ExtraTreesClassifier 

In [12]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,train_input,train_target,return_train_score=True,n_jobs=-1)
print(np.mean(scores['train_score']),np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [13]:
et.fit(train_input,train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]
